In [ ]:
!pip install -q torch torchvision tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.9 MB/s eta 0:00:00


In [ ]:
import kagglehub
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

path = kagglehub.dataset_download("dansbecker/food-101")

In [ ]:
# 2. 경로 설정
train_dir = '/kaggle/input/food-101/food-101/food-101/images'
# 이 경로에는 101개의 폴더가 있으며, 각 폴더는 하나의 음식 카테고리를 나타냅니다.

# 3. 하이퍼파라미터
batch_size = 32
num_epochs = 20
learning_rate = 0.001
num_classes = 101
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 4. 전처리 및 데이터로더
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Food-101에는 train/val 분할이 따로 없으므로 일부 데이터를 검증용으로 나눠야 합니다.
dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 5. 모델 준비 (ResNet50 + Fine-tuning)
model = models.resnet50(pretrained=True)


for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 201MB/s]


In [ ]:
start_finetune_epoch = 11  # Epoch 11부터 fine-tuning 시작

In [ ]:
# 6. 학습
for epoch in range(num_epochs):

    # epoch 11 진입 시점에서 fine-tuning 적용
    if epoch == start_finetune_epoch:
        print(f"\n🔧 Fine-Tuning 시작 (Epoch {epoch})...\n")
        for param in model.parameters():
            param.requires_grad = True 

        optimizer = optim.Adam(model.parameters(), lr=1e-5) 

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # 검증
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (preds == labels).sum().item()

    val_loss = val_loss / val_total
    val_acc = 100 * val_correct / val_total
    print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

# 7. 모델 저장
torch.save(model.state_dict(), "resnet50_food101_epoch20.pth")

Epoch 1 [Train]: 100%|██████████| 2841/2841 [14:43<00:00,  3.22it/s]


Train Loss: 2.3269, Accuracy: 43.96%


Epoch 1 [Val]: 100%|██████████| 316/316 [01:37<00:00,  3.23it/s]


Val Loss: 1.9868, Accuracy: 51.73%


Epoch 2 [Train]: 100%|██████████| 2841/2841 [08:20<00:00,  5.67it/s]


...

Val Loss: 1.0728, Accuracy: 75.54%


Epoch 20 [Train]: 100%|██████████| 2841/2841 [10:38<00:00,  4.45it/s]


Train Loss: 0.0723, Accuracy: 98.61%


Epoch 20 [Val]: 100%|██████████| 316/316 [00:55<00:00,  5.67it/s]


Val Loss: 1.0841, Accuracy: 75.95%
